In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
fake = pd.read_csv('C:\\Users\\snipe\\FND Project\\Fake.csv',encoding='latin1') 
true = pd.read_csv('C:\\Users\\snipe\\FND Project\\True.csv',encoding='latin1')

In [3]:
fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-Dec-17
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-Dec-17
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-Dec-17
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-Dec-17
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-Dec-17


In [5]:
fake['target'] = 'FAKE'
true['target'] = 'REAL'

In [6]:
data = pd.concat([fake, true]).reset_index(drop = True)
data

,title,text,subject,date,target
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",FAKE
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",FAKE
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,22-Aug-17,REAL
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,22-Aug-17,REAL
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,22-Aug-17,REAL
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,22-Aug-17,REAL


In [7]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,title,text,subject,date,target
0,U.S. urges Iraq to avoid clashes with Kurds ne...,WASHINGTON (Reuters) - The U.S. State Departme...,worldnews,20-Oct-17,REAL
1,Catalan independence campaign kicks off as Bar...,"TARRAGONA, Spain (Reuters) - The Catalonian go...",worldnews,14-Sep-17,REAL
2,Islamic State claims responsibility for attack...,CAIRO (Reuters) - Islamic State has claimed re...,worldnews,07-Nov-17,REAL
3,W.H. Admits Trump Lied: Never âWitnessed Ho...,Donald Trump claimed on Wednesday morning that...,News,"August 31, 2017",FAKE
4,Trumpâs Lawyerâs Response Was Riddled Wit...,Donald Trump hired himself a personal lawyer t...,News,"June 8, 2017",FAKE


In [8]:
data.drop(["date"],axis=1,inplace=True)
data.drop(["title"],axis=1,inplace=True)
data

,text,subject,target
0,WASHINGTON (Reuters) - The U.S. State Departme...,worldnews,REAL
1,"TARRAGONA, Spain (Reuters) - The Catalonian go...",worldnews,REAL
2,CAIRO (Reuters) - Islamic State has claimed re...,worldnews,REAL
3,Donald Trump claimed on Wednesday morning that...,News,FAKE
4,Donald Trump hired himself a personal lawyer t...,News,FAKE
...,...,...,...
44893,LIMA (Reuters) - Peruvian President Pedro Pabl...,worldnews,REAL
44894,KABUL (Reuters) - A car bomber attacked a Dani...,worldnews,REAL
44895,Hillary Clinton today falsely called Donald Tr...,politics,FAKE
44896,The left has officially taken parenting out of...,left-news,FAKE


In [9]:
data.isnull().any()

text       False
subject    False
target     False
dtype: bool

In [10]:
data['text'] = data['text'].apply(lambda x: x.lower())
data

,text,subject,target
0,washington (reuters) - the u.s. state departme...,worldnews,REAL
1,"tarragona, spain (reuters) - the catalonian go...",worldnews,REAL
2,cairo (reuters) - islamic state has claimed re...,worldnews,REAL
3,donald trump claimed on wednesday morning that...,News,FAKE
4,donald trump hired himself a personal lawyer t...,News,FAKE
...,...,...,...
44893,lima (reuters) - peruvian president pedro pabl...,worldnews,REAL
44894,kabul (reuters) - a car bomber attacked a dani...,worldnews,REAL
44895,hillary clinton today falsely called donald tr...,politics,FAKE
44896,the left has officially taken parenting out of...,left-news,FAKE


In [11]:
import string
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
data['text'] = data['text'].apply(punctuation_removal)
data

,text,subject,target
0,washington reuters the us state department on...,worldnews,REAL
1,tarragona spain reuters the catalonian govern...,worldnews,REAL
2,cairo reuters islamic state has claimed respo...,worldnews,REAL
3,donald trump claimed on wednesday morning that...,News,FAKE
4,donald trump hired himself a personal lawyer t...,News,FAKE
...,...,...,...
44893,lima reuters peruvian president pedro pablo k...,worldnews,REAL
44894,kabul reuters a car bomber attacked a danish ...,worldnews,REAL
44895,hillary clinton today falsely called donald tr...,politics,FAKE
44896,the left has officially taken parenting out of...,left-news,FAKE


In [12]:
#stopwords removal
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head(50)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HZT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,subject,target
0,washington reuters us state department friday ...,worldnews,REAL
1,tarragona spain reuters catalonian government ...,worldnews,REAL
2,cairo reuters islamic state claimed responsibi...,worldnews,REAL
3,donald trump claimed wednesday morning witness...,News,FAKE
4,donald trump hired personal lawyer represent r...,News,FAKE
5,palm beach fla reuters president donald trump ...,politicsNews,REAL
6,republican presidential frontrunner donald tru...,News,FAKE
7,brussels reuters european commission said deep...,politicsNews,REAL
8,washington reuters us president donald trump r...,politicsNews,REAL
9,trey gowdy rips clinton case using private ema...,politics,FAKE


In [25]:
x_train,x_test,y_train,y_test = train_test_split(data['text'], data.target, test_size=0.2)

In [26]:
#tfidf_vectorizer
tfidf_vectorizer=TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train=tfidf_vectorizer.fit_transform(x_train) 
tfidf_test=tfidf_vectorizer.transform(x_test)

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),
                    ('rnmodel', RandomForestClassifier())])

In [28]:
modelrn = pipeline.fit(x_train,y_train)

C:\Users\HZT\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [29]:
prediction = modelrn.predict(x_test)
print("Accuracy: {}%".format(round(accuracy_score(y_test, prediction)*100,2)))

Accuracy: 97.58%


In [31]:
pred = modelrn.predict(x_test)

In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

        FAKE       0.97      0.98      0.98      4757
        REAL       0.98      0.97      0.97      4223

    accuracy                           0.98      8980
   macro avg       0.98      0.98      0.98      8980
weighted avg       0.98      0.98      0.98      8980



In [33]:
print(confusion_matrix(y_test, pred))

[[4676   81]
 [ 136 4087]]


In [34]:
pickle.dump(modelrn, open('modelrn.pkl','wb'))  
modelrn = pickle.load(open('modelrn.pkl','rb'))

In [36]:
print(modelrn.predict(["How elite leftists celebrate America s Independence Day The Kennedy clan gathered at their Hyannis Port compound on Cape Cod over the weekend for their annual Fourth of July festivities, and took some time to attack Donald Trump.Robert F. Kennedy Jr. s daughter Kathleen, between known as Kick, posted a photos of a pinata of The Donald from a family party over the weekend. It s yuge party!,  wrote Kick in the caption of the Instagram post, which also showed some of her family members milling about in the background.She later deleted the Instagram post just before 11am on Monday.Via: Daily Mail"]))

['FAKE']
